In [2]:
import pandas as pd 
import numpy as np

df  = pd.read_csv("raw_data.csv", index_col=0)
cleandf = pd.DataFrame(columns=['expNum','age','gender','ingroupNorm','bothShown','ingroupFirst','response','rating',
                                'understandingCheckResponse','understandingCheckOrder','responseTime','understandingCheckTime',
                                'elapsedTime','utcTime','RepublicanRating','DemocraticRating','topIssue','topIssueRating',
                                 'identityIngroupResponse','identityOutgroupResponse','ingroupAgree','outgroupDisagree'])
# In the dataset, every participant has 7 lines of data
firstlndf = df.iloc[0::7, :].reset_index() # first one has topIssue
secondlndf = df.iloc[1::7, :].reset_index() # second one has topIssueRating
thirdlndf = df.iloc[2::7, :].reset_index() # third one has response
fourthlndf = df.iloc[3::7, :].reset_index() # fourth one has rating
fifthlndf = df.iloc[4::7, :].reset_index() # fifth one has understandingCheckResponse
sixthlndf = df.iloc[5::7, :].reset_index() # sixth one has identityIngroupResponse
seventhlndf = df.iloc[6::7, :].reset_index() # seventh one has identityOutgroupResponse

fifthlndf['response'] = fifthlndf['response'].replace({'Participants chose which action they preferred':1,
                                          'Due to a computer error, participants were not allocated equally to imagine performing the different actions':2,
                                          'No data was saved during the experiment.':3,
                                          'The participants completed the experiment with their eyes closed.':4})


cleandf['age'] = firstlndf['age']
cleandf['expNum'] = 'Exp 1'
cleandf['gender'] = firstlndf['gender']
cleandf['ingroupNorm'] = firstlndf['ingroup_descriptive_norm']
cleandf['ingroupNorm'] = np.where((cleandf['ingroupNorm']) == -1, 0 , cleandf['ingroupNorm'])
cleandf['bothShown'] = firstlndf['both_norms_shown']
cleandf['ingroupFirst'] = 0
cleandf['response'] =  pd.to_numeric(thirdlndf['response'])
cleandf['rating'] = fourthlndf['response']
cleandf['understandingCheckResponse'] = fifthlndf['response']
cleandf['understandingCheckOrder'] = 1234
cleandf['responseTime'] = firstlndf['timeSpent'].mean()  / 7
cleandf['understandingCheckTime']  = firstlndf['timeSpent'].mean()  / 7
cleandf['elapsedTime']  = firstlndf['timeSpent'].mean() / 7
cleandf['utcTime'] = firstlndf['startDate']
cleandf['topIssue'] = firstlndf['response']
cleandf['topIssueRating'] = pd.to_numeric(secondlndf['response']) + 5
cleandf['identityIngroupResponse'] = pd.to_numeric(sixthlndf['response'])
cleandf['identityOutgroupResponse'] = pd.to_numeric(seventhlndf['response'])
cleandf['response'] += 4
cleandf['identityIngroupResponse'] += 4
cleandf['identityOutgroupResponse'] += 4

# keep only valid participants
cleandf = cleandf.loc[cleandf['understandingCheckResponse'] == 1]

# add ingroup agree in binary value, 1 if >= 5, else 0
cleandf['ingroupAgree'] = np.where((cleandf['identityIngroupResponse'] >= 5), 1,cleandf['ingroupAgree'])
cleandf['ingroupAgree'] = np.where((cleandf['identityIngroupResponse'] < 5), 0,cleandf['ingroupAgree'])

# add outgroup disagree in binary value, 1 if >= 5, else 0
cleandf['outgroupDisagree'] = np.where((cleandf['identityOutgroupResponse'] < 4), 1,cleandf['outgroupDisagree'])
cleandf['outgroupDisagree'] = np.where((cleandf['identityOutgroupResponse'] > 3), 0,cleandf['outgroupDisagree'])

# there is not really a response value 4, it is just -3, -2, -1, 1, 2, 3; so it should be in the range of 1-6. 1-3 can stay the same, 5-7 have to be minus one
cleandf['response'] = np.where((cleandf['response'] == 5), 4,cleandf['response'])
cleandf['response'] = np.where((cleandf['response'] == 6), 5,cleandf['response'])
cleandf['response'] = np.where((cleandf['response'] == 7), 6,cleandf['response'])

# save dataframe
cleandf.to_csv('clean_data.csv')

# shorter dataframe with only relevant values
#shortdf = cleandf[['ingroupNorm','bothShown','response','ingroupAgree','outgroupDisagree']]
#shortdf.to_csv('short_data.csv')
#shortdf
cleandf

,expNum,age,gender,ingroupNorm,bothShown,ingroupFirst,response,rating,understandingCheckResponse,understandingCheckOrder,...,elapsedTime,utcTime,RepublicanRating,DemocraticRating,topIssue,topIssueRating,identityIngroupResponse,identityOutgroupResponse,ingroupAgree,outgroupDisagree
0,Exp 1,22.0,male,0,0,0,6,3,1,1234,...,0.25814,Tue Aug 24 2021 18:16:54 GMT+0200 (Central Eur...,NaN,NaN,Climate change,10,5,1,1,1
1,Exp 1,19.0,male,1,1,0,6,2,1,1234,...,0.25814,Mon Aug 23 2021 20:06:42 GMT+0200 (Mitteleurop...,NaN,NaN,Drug legalization,8,7,2,1,1
3,Exp 1,22.0,female,1,0,0,2,1,1,1234,...,0.25814,Mon Aug 23 2021 11:30:20 GMT+0200 (CEST),NaN,NaN,Climate change,10,7,1,1,1
4,Exp 1,60.0,female,1,1,0,2,-1,1,1234,...,0.25814,Sun Aug 22 2021 19:42:26 GMT+0200 (Mitteleurop...,NaN,NaN,Climate change,9,2,1,0,1
5,Exp 1,27.0,male,1,0,0,6,1,1,1234,...,0.25814,Sun Aug 22 2021 19:06:49 GMT+0200 (CEST),NaN,NaN,Climate change,10,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Exp 1,19.0,male,0,1,0,4,1,1,1234,...,0.25814,Fri Aug 13 2021 16:23:08 GMT+0200 (Mitteleurop...,NaN,NaN,The Black Lives Matter movement,0,5,1,1,1
93,Exp 1,21.0,female,1,1,0,3,1,1,1234,...,0.25814,Fri Aug 13 2021 16:22:29 GMT+0200 (Ora legale ...,NaN,NaN,Feminism,10,7,1,1,1
94,Exp 1,20.0,male,1,0,0,1,3,1,1234,...,0.25814,Thu Aug 12 2021 20:48:02 GMT+0200 (Mitteleurop...,NaN,NaN,Joe Biden,7,5,5,1,0
96,Exp 1,20.0,male,1,0,0,1,2,1,1234,...,0.25814,Mon Jul 26 2021 21:28:12 GMT+0200 (Mitteleurop...,NaN,NaN,The Black Lives Matter movement,1,6,1,1,1
